<a href="https://colab.research.google.com/github/smfwlgus/DnM/blob/master/NongNetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import urllib.request
import json
import datetime

# **어제의 농넷 데이터 불러오기** 

In [ ]:
#어제의 농넷 데이터 불러오기
#datetime.datetime(2019, 12, 12)...
today = datetime.datetime.today()
yesterday = (today-datetime.timedelta(1)).strftime('%Y%m%d') #strftime(): 원하는 날짜 형식으로 출력

url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

response = urllib.request.urlopen(url+yesterday).read()
response = json.loads(response)

data = pd.DataFrame(response['data'])

In [ ]:
data

,PUM_NM,LV_NM,TOT_AMT,SAN_NM,SALEDATE,CMP_NM,DAN_NM,WHSAL_NM,SIZE_NM,COST,POJ_NM,TOT_QTY,QTY,KIND_NM,DANQ
0,포도,특,351000,경상북도 김천시,20211011,덕풍청과,kg,인천구월도매,.,13000,상자,54.0,27.0,샤인마스캇,2.0
1,포도,특,1100000,경상북도 김천시,20211011,덕풍청과,kg,인천구월도매,.,11000,상자,200.0,100.0,샤인마스캇,2.0
2,포도,특,225000,경상북도 김천시,20211011,덕풍청과,kg,인천구월도매,.,9000,상자,50.0,25.0,샤인마스캇,2.0
3,포도,특,440000,경상북도 경산시,20211011,덕풍청과,kg,인천구월도매,.,11000,상자,120.0,40.0,마스캇베리에이,3.0
4,포도,특,230000,경상북도 경산시,20211011,덕풍청과,kg,인천구월도매,.,11500,상자,60.0,20.0,마스캇베리에이,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122346,바나나,.,170000,None,20211011,강서청과,kg,서울강서도매,1개(내_뿌리),17000,.,130.0,10.0,바나나(수입),13.0
122347,바나나,.,170000,None,20211011,강서청과,kg,서울강서도매,1개(내_뿌리),17000,.,130.0,10.0,바나나(수입),13.0
122348,바나나,.,170000,None,20211011,강서청과,kg,서울강서도매,1개(내_뿌리),17000,.,130.0,10.0,바나나(수입),13.0
122349,바나나,.,85000,None,20211011,강서청과,kg,서울강서도매,1개(내_뿌리),17000,.,65.0,5.0,바나나(수입),13.0


In [ ]:
# data.to_csv('ddd.csv')

# **일자(하루) 데이터 가져오기**

In [ ]:
#일자 하루
nalja = datetime.datetime(2021, 8, 24)
august = nalja.strftime('%Y%m%d') #strftime(): 원하는 날짜 형식으로 출력

url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

response = urllib.request.urlopen(url+august).read()
response = json.loads(response)

data = pd.DataFrame(response['data'])

In [ ]:
nalja = datetime.datetime(2021, 8, 24)
abc = nalja.strftime('%Y%m%d') #strftime(): 원하는 날짜 형식으로 출력

url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

response = urllib.request.urlopen(url+abc).read()
response = json.loads(response)

data = pd.DataFrame(response['data'])

nalja = datetime.datetime(2021, 8, 25)
august = nalja.strftime('%Y%m%d') #strftime(): 원하는 날짜 형식으로 출력

url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

response = urllib.request.urlopen(url+august).read()
response = json.loads(response)

df1 = data.append(response['data'])

In [ ]:
# df1.to_csv('dskldj.csv', index=False)

# 1~7월

In [ ]:
#1~7월 반복문 실험

for i in range(1, 3):
  nalja = datetime.datetime(2021, 12, i)
  nalja2 = nalja.strftime('%Y%m%d')

  #nalja3 = nalja.strftime('%Y%m')

  url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜
  
  response = urllib.request.urlopen(url+nalja2).read()
  response = json.loads(response)
  
  globals() ['df{}'.format(i)] = pd.DataFrame(response['data'])

In [ ]:
august = pd.concat([df1, df2])

In [ ]:
august = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, 
                    df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,
                    df21, df22, df23, df24, df25, df26, df27, df28, df29, df30])

In [ ]:
august.to_csv('/content/drive/MyDrive/Capstone/Data/AT_TSALET_ALL_202112.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import multiprocessing
import pickle

In [ ]:
# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '마늘', '대파', 
        '깻잎', '시금치', '미나리', '파프리카', '토마토',
    ]
    
    t_dict = {
        'date':[]
    }
    
    for sub in unique_pum:
        t_dict[f'{sub}_거래량(kg)'] = []
        t_dict[f'{sub}_가격(원/kg)'] = []
        
    tsalet_sample = pd.read_csv(tsalet_file)
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        t_dict['date'].append(day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>=0)]
            if c.shape[0] == 0:
                t_dict[f'{sub}_거래량(kg)'].append(0)
                t_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                t_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                t_dict[f'{sub}_가격(원/kg)'].append(mean_price)
                
    with open(f'/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/{tsalet_file.split("/")[-1].split(".")[0]}.pkl', 'wb') as f:
        pickle.dump(t_dict, f)

In [ ]:
#tsalet_files = sorted(glob('/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/*'))
tsalet_files = sorted(glob('/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/AT_TSALET_ALL_202112.csv'))


pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
pool.map(preprocessing, tsalet_files)
pool.close()
pool.join()

In [ ]:
dict_files = sorted(glob('/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/AT_TSALET_ALL_202112.pkl'))

In [ ]:
dict_files

['/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/AT_TSALET_ALL_202112.pkl']

In [ ]:
t_dict_list = []
for dict_file in dict_files:
    with open(dict_file, 'rb') as f:
        train_dict = pickle.load(f)
    t_dict_list.append(train_dict)

In [ ]:
train = None

for t_dcit in t_dict_list:
    if train is None:
        train = pd.DataFrame(t_dcit)
    else:
        train = pd.concat([train, pd.DataFrame(t_dcit)])

In [ ]:
train['date'] = train.date.astype(str).str.replace('-','')
train['date'] = pd.to_datetime(train.date, format='%Y%m%d')

In [ ]:
train.insert(1, '요일', '0')

In [ ]:
import datetime #20/9/29~20/11/5
days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']

for i in range (train.shape[0]): #문자 -> 숫자
  train['요일'][i] = days[int(train['date'].dt.weekday[i])]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
train['date'] = train['date'].dt.date

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Capstone/Data/test.csv')
test

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
0,2021-01-01,금요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,2021-01-02,토요일,24401.50,388.534188,21461.0,559.526583,20606.0,1058.393186,2408.0,5129.931063,27859.0,2384.020245,4079.5,13150.276995,9920.00,2642.957863,7033.0,1614.409498,7849.0,4884.721875,25290.0,1914.895216
2,2021-01-03,일요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,2021-01-04,월요일,859651.80,441.499637,1665261.7,542.826183,1823037.4,1266.627437,121200.4,5425.898627,874594.6,2202.568178,112324.7,9501.869972,374131.63,2966.183666,79020.5,3509.511798,150290.6,4515.337752,393979.0,2098.278649
4,2021-01-05,화요일,889332.20,412.451877,1647835.5,570.874791,1513826.0,1329.721668,125712.0,5147.233605,765322.2,2240.754967,69850.2,11128.609839,295523.20,3111.929321,74112.0,3965.334750,89124.0,5515.671581,228576.5,2092.442574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2021-11-26,금요일,3322119.50,855.526302,2848866.7,435.322178,1420063.1,824.988797,77781.7,6688.693844,606203.2,1159.468442,67183.0,5270.268357,229238.05,2226.226340,82529.9,6738.824341,165520.6,2108.242279,231763.5,2675.807783
330,2021-11-27,토요일,3099051.25,844.090363,2657614.8,378.614620,1354649.4,772.076177,77304.3,6706.403292,713593.0,1086.698594,67172.0,5169.389046,221894.62,2275.984041,85130.5,8013.229947,122186.3,2080.165043,196120.0,2294.284622
331,2021-11-28,일요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
332,2021-11-29,월요일,2856016.45,801.194158,2209466.0,380.848191,1910143.7,816.013853,84870.7,6332.339559,682166.3,1150.958392,79927.8,5897.250343,275299.90,2294.806424,82265.7,7140.467011,203478.4,2274.810506,232403.5,2486.894345


In [ ]:
test = test.append(train)
test

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
0,2021-01-01,금요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,2021-01-02,토요일,24401.50,388.534188,21461.0,559.526583,20606.0,1058.393186,2408.0,5129.931063,27859.0,2384.020245,4079.5,13150.276995,9920.00,2642.957863,7033.0,1614.409498,7849.0,4884.721875,25290.0,1914.895216
2,2021-01-03,일요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,2021-01-04,월요일,859651.80,441.499637,1665261.7,542.826183,1823037.4,1266.627437,121200.4,5425.898627,874594.6,2202.568178,112324.7,9501.869972,374131.63,2966.183666,79020.5,3509.511798,150290.6,4515.337752,393979.0,2098.278649
4,2021-01-05,화요일,889332.20,412.451877,1647835.5,570.874791,1513826.0,1329.721668,125712.0,5147.233605,765322.2,2240.754967,69850.2,11128.609839,295523.20,3111.929321,74112.0,3965.334750,89124.0,5515.671581,228576.5,2092.442574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2021-11-28,일요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
332,2021-11-29,월요일,2856016.45,801.194158,2209466.0,380.848191,1910143.7,816.013853,84870.7,6332.339559,682166.3,1150.958392,79927.8,5897.250343,275299.90,2294.806424,82265.7,7140.467011,203478.4,2274.810506,232403.5,2486.894345
333,2021-11-30,화요일,2185300.05,733.173048,1718213.9,418.116762,1494509.0,753.684039,120348.6,6278.178749,578445.3,1249.963497,56564.5,6505.918429,231467.55,2242.437702,75398.8,6544.052319,159247.3,2225.523196,176793.4,2358.261281
0,2021-12-01,수요일,1517157.50,814.134951,1515337.4,355.843013,1713024.7,804.114206,159602.7,6826.912909,513202.7,1398.334826,61748.9,6717.704607,220261.97,2417.830949,86655.6,6068.800828,162380.7,2134.811434,190878.7,2507.076484


In [ ]:
test.to_csv('/content/drive/MyDrive/Capstone/Data/test_yesterday.csv', index=False)

# <font color = 'skyblue'> **8, 9, 10월 데이터** <font>

In [ ]:
#8월 반복문 실험

for i in range(1, 32):
  nalja = datetime.datetime(2021, 8, i)
  nalja2 = nalja.strftime('%Y%m%d')

  #nalja3 = nalja.strftime('%Y%m')

  url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜
  
  response = urllib.request.urlopen(url+nalja2).read()
  response = json.loads(response)
  
  globals() ['df{}'.format(i)] = pd.DataFrame(response['data'])

In [ ]:
august = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, 
                    df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,
                    df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31])

In [ ]:
august.to_csv('/content/drive/MyDrive/Capstone/Data/202108.csv', index=False)

# 9월

In [ ]:
#9월 반복문

for i in range(28, 31):
  nalja = datetime.datetime(2021, 9, i)
  nalja2 = nalja.strftime('%Y%m%d')

  #nalja3 = nalja.strftime('%Y%m')

  url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜
  
  response = urllib.request.urlopen(url+nalja2).read()
  response = json.loads(response)
  
  globals() ['sep{}'.format(i)] = pd.DataFrame(response['data'])

In [ ]:
df = pd.DataFrame()

for i in range(1, 31):
      df = pd.concat([df, globals() ['sep{}'.format(i)]])

In [ ]:
df

,PUM_NM,LV_NM,TOT_AMT,SAN_NM,SALEDATE,CMP_NM,DAN_NM,WHSAL_NM,SIZE_NM,COST,POJ_NM,TOT_QTY,QTY,KIND_NM,DANQ
0,바나나,특,288000.0,필리핀,20210901,안양농산물(주),kg,안양도매시장,.,24000.0,상자,156.0,12.0,기타바나나,13.0
1,참외,특,28000.0,경북 안동시,20210901,포항청과,kg,포항도매시장,.,14000.0,상자,20.0,2.0,기타참외,10.0
2,참외,상,28400.0,경북 안동시,20210901,포항청과,kg,포항도매시장,.,7100.0,상자,40.0,4.0,기타참외,10.0
3,참외,특,7800.0,경북 안동시,20210901,포항청과,kg,포항도매시장,.,7800.0,상자,10.0,1.0,기타참외,10.0
4,참외,특,10200.0,경북 안동시,20210901,포항청과,kg,포항도매시장,.,10200.0,상자,10.0,1.0,기타참외,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102421,바나나,.,600000.0,None,20210930,강서청과,kg,서울강서도매,1개(내_뿌리),20000.0,.,390.0,30.0,바나나(수입),13.0
102422,바나나,.,100000.0,None,20210930,강서청과,kg,서울강서도매,1개(내_뿌리),20000.0,.,65.0,5.0,바나나(수입),13.0
102423,바나나,.,100000.0,None,20210930,강서청과,kg,서울강서도매,1개(내_뿌리),20000.0,.,65.0,5.0,바나나(수입),13.0
102424,바나나,.,100000.0,None,20210930,강서청과,kg,서울강서도매,1개(내_뿌리),20000.0,.,65.0,5.0,바나나(수입),13.0


In [ ]:
september = pd.concat([sep1, sep2, sep3, sep4, sep5, sep6, sep7, sep8, sep9, sep10,
                       sep11, sep12, sep13, sep14, sep15, sep16, sep17, sep18, sep19, sep20,
                       sep21, sep22, sep23, sep24, sep25, sep26, sep27, sep28, sep29, sep30])

# september.to_csv('202109.csv', index=False)

In [ ]:
september = pd.concat([sep28, sep29, sep30])

september.to_csv('202109.csv', index=False)

# 10월

In [ ]:
#10월 반복문

for i in range(1, 32):
  nalja = datetime.datetime(2021, 10, i)
  nalja2 = nalja.strftime('%Y%m%d')

  #nalja3 = nalja.strftime('%Y%m')

  url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜
  
  response = urllib.request.urlopen(url+nalja2).read()
  response = json.loads(response)
  
  globals() ['oct{}'.format(i)] = pd.DataFrame(response['data'])

In [ ]:
october = pd.concat([oct1, oct2, oct3, oct4, oct5, oct6, oct7, oct8, oct9, oct10,
                     oct11, oct12, oct13, oct14, oct15, oct16, oct17, oct18, oct19, oct20,
                     oct21, oct22, oct23, oct24, oct25, oct26, oct27, oct28, oct29, oct30, oct31])

october.to_csv('/content/drive/MyDrive/Capstone/Data/202110.csv', index=False)

# <font color = 'skyblue'> **월 별 데이터 함수** <font>

* test_AT_TSALET_ALL 데이터 수집 코드
* 수정필요


In [ ]:
#월 별 농산물 데이터 가져오기
def getMonthData(month, start, finish):
  for i in range(start, finish):
    nalja = datetime.datetime(2021, month, i).strftime('%Y%m%d')

    url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜
  
    response = urllib.request.urlopen(url+nalja).read()
    response = json.loads(response)

    globals() ['df{}'.format(i)] = pd.DataFrame(response['data'])  
    
    df = pd.DataFrame()

    for i in range(start, finish):
      df = df.append(f'df{i}')
  
    return df 

In [ ]:
#september = getMonthData(9, 28, 31)
#september

In [ ]:
#september.to_csv('september.csv', index = False)

In [ ]:
october = getMonthData(10, 1, 3)
october

TypeError: ignored

In [ ]:
october.to_csv('october.csv', index = False)